## Select Environment & Start Date

In [ ]:
env = "PAPER"
start_date = '2020-10-01'

#### Imports

In [ ]:
import liualgotrader.analytics.analysis as ana
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
# silence warnings
import warnings
warnings.filterwarnings('ignore')

import pyfolio as pf
%matplotlib inline
import nest_asyncio

nest_asyncio.apply()

#### Load Portfolio data from database

In [ ]:
stock_rets = ana.portfolio_return(env, start_date)

In [ ]:
stock_rets

## Show Portfolio analysis

In [ ]:
with plt.xkcd():
    for column in stock_rets:
        plt.plot(stock_rets[column])
        plt.ylabel(column)
        plt.xticks(rotation=45) 
        plt.show()